<a href="https://colab.research.google.com/github/cclljj/Google_colab_ipynb/blob/master/Use_Python_to_Take_URL_Screenshot_and_Send_Message_to_Line_Notify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網頁畫面批次擷取工具

功能簡述：
- 根據輸入的 URL 列表，擷取每一個 URL 的瀏覽器瀏覽畫面圖片
- 根據設定的排列方式，將所有產生的圖片整併成一張大圖
- 將產製出來的整併圖，送至預先設定好的 Line Notify 帳號

In [ ]:
# 安裝相關函式庫
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
# 匯入相關函式庫
from selenium import webdriver
from time import sleep 
from PIL import Image
import numpy as np
from datetime import datetime
import requests

# 初始化設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--window-size=1280x1024")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-certificate-errors-spki-list')
options.add_argument('--ignore-ssl-errors')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
# ====================================
# 請根據需求修改此段落的相關設定

# 請依照範例設定所需要產製的網站畫面網址列表

urls = [
        "https://www.google.com",
        "https://www.microsoft.com",
        "https://www.apple.com",
        "https://www.amazon.com",
]

# LINE Notify 權杖
Line_tokens = ['your_line_notify_token_1', # 1st token
               'your_line_notify_token_2', # 2st token
]

# 最後輸出檔的檔案名稱
fname_out = "output.jpg"
fname_out_resize = "output_resize.jpg"

# 圖片中每一列的網站數目
num_of_col = 5

# ====================================

In [ ]:
# 依照輸入的網址列表，依序產製個別的網站截圖

fnames = []

wd = webdriver.Chrome(options=options)
wd.set_page_load_timeout(30)
for i in range(0, len(urls)):
  try:
    wd.get(urls[i])
    sleep(1)
    wd.get_screenshot_as_file(str(i) + ".png") 
    print(i+1, "/", len(urls), urls[i], "Success!")
    fnames.append(str(i) + ".png")
  except Exception as e:
    print(i,urls[i], "Error!")
    print(e)

if len(fnames) % num_of_col:
  white_img = Image.new('RGB', (1280, 1024), color = (255,255,255))
  white_img.save("white.png")
  for i in range(0, num_of_col - len(fnames) % num_of_col):
    fnames.append("white.png")
  

In [ ]:
# 將個別的網站截圖彙整成一張大圖

def pil_grid(images, max_horiz=np.iinfo(int).max):
    n_images = len(images)
    n_horiz = min(n_images, max_horiz)
    h_sizes, v_sizes = [0] * n_horiz, [0] * (n_images // n_horiz)
    for i, im in enumerate(images):
        h, v = i % n_horiz, i // n_horiz
        h_sizes[h] = max(h_sizes[h], im.size[0])
        v_sizes[v] = max(v_sizes[v], im.size[1])
    h_sizes, v_sizes = np.cumsum([0] + h_sizes), np.cumsum([0] + v_sizes)
    im_grid = Image.new('RGB', (h_sizes[-1], v_sizes[-1]), color='white')
    for i, im in enumerate(images):
        im_grid.paste(im, (h_sizes[i % n_horiz], v_sizes[i // n_horiz]))
    return im_grid

imgs    = [ Image.open(i) for i in fnames ]
final_img = pil_grid(imgs, num_of_col)
final_img.save(fname_out)
#final_img

In [ ]:
resize_img = final_img.resize((1024,1024))
resize_img.save(fname_out_resize)

# 要發送的訊息
now = datetime.now()
message = now.strftime("%d/%m/%Y %H:%M:%S")

for Line_token in Line_tokens:
  # HTTP 標頭參數與資料
  headers = { "Authorization": "Bearer " + Line_token }
  data = { 'message': message }

  # 要傳送的圖片檔案
  image = open(fname_out_resize, 'rb')
  files = { 'imageFile': image }

  # 以 requests 發送 POST 請求
  requests.post("https://notify-api.line.me/api/notify",
    headers = headers, data = data, files = files)

In [ ]:
# 將過程中產製的中繼圖片刪除
for fname in fnames:
  if os.path.exists(fname):
    os.remove(fname)

# Reference
- [Use selenium in Colab](https://gist.github.com/korakot/5c8e21a5af63966d80a676af0ce15067)
- [Combine several images horizontally with Python - Stack Overflow](https://stackoverflow.com/questions/30227466/combine-several-images-horizontally-with-python)
- [Python 以 LINE Notify 自動傳送即時訊息、圖片教學與範例 - Office 指南](https://officeguide.cc/python-line-notify-send-messages-images-tutorial-examples/)